# 여러 문서에서 찾아서 답변하는 챗봇 만들기

> 유튜브 [빵형의 개발도상국](https://www.youtube.com/@bbanghyong)

- QA ChatBot
- LangChain
- ChatGPT (gpt-3.5-turbo)
- ChromaDB

In [ ]:
!pip install -q langchain openai tiktoken chromadb gdown
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 813.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

## PDF to Text
압축되어 있는 여러 PDF 파일을 압축해제 한 뒤, 텍스트로 변환

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import zipfile
# import os

# # 압축 해제할 파일 경로
# zip_file_path = r'/content/drive/MyDrive/QUANTIFY/경제동향 자료.zip'

# # 압축 해제될 폴더 경로
# output_folder = r'/content/drive/MyDrive/QUANTIFY/google_drive_file.zip'

# # 폴더가 없다면 생성
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # Zip 파일 열기 및 압축 해제
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(output_folder)

# print(f'Files extracted to {output_folder}')

Files extracted to /content/drive/MyDrive/QUANTIFY/google_drive_file.zip


In [ ]:
# from pdfminer.high_level import extract_text

# # PDF 파일을 저장한 폴더의 경로
# pdf_folder = r'/content/drive/MyDrive/QUANTIFY/google_drive_file.zip'  # 압축 해제된 PDF 파일이 있는 폴더 경로
# output_text_folder = r'/content/drive/MyDrive/QUANTIFY/text_files'  # 텍스트 파일을 저장할 폴더 경로

# import os

# if not os.path.exists(output_text_folder):
#     os.makedirs(output_text_folder)

# # 폴더 내의 모든 PDF 파일에 대해 반복
# for pdf_filename in os.listdir(pdf_folder):
#     if pdf_filename.endswith(".pdf"):
#         pdf_path = os.path.join(pdf_folder, pdf_filename)

#         # PDF를 텍스트로 추출
#         text = extract_text(pdf_path)

#         # 텍스트를 텍스트 파일로 저장
#         txt_filename = pdf_filename.replace(".pdf", ".txt")
#         txt_path = os.path.join(output_text_folder, txt_filename)
#         with open(txt_path, "w", encoding="utf-8") as txt_file:
#             txt_file.write(text)

## Setting up LangChain

OpenAI API Key

https://platform.openai.com/account/api-keys

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI_API')

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

## Load multiple and process documents

In [ ]:
# loader = TextLoader('single_text_file.txt')
output_text_folder = r'/content/drive/MyDrive/QUANTIFY/text_files'  # 텍스트 파일을 저장한 폴더 경로 - 신문 및 보고서 txt 파일 모음집(경로 수정 필요)
loader = DirectoryLoader(output_text_folder, glob="*.txt", loader_cls=TextLoader)

documents = loader.load()

len(documents)

13

In [ ]:
documents

## Split texts

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

len(texts)

3438

In [ ]:
texts[2:4]

[Document(page_content='6-2. 경상수지 및 금융계정 ······································································································ 45\n\nCurrent\x00 Account\x00 and\x00 Financial\x00 Account\n\n7. 고용 및 임금 ····················································································································· 47\n\nEmployment\x00 and\x00 Wage\n\n8. 물가 ····································································································································· 49\n\nPrice\x00 Indices\n\n9-1. 금융시장 (I) ······················································································································· 51\n\nFinancial\x00 Market\x00 (I)\n\n9-2. 금융시장 (II) ······················································································································ 53\n\nFinancial\x00 Market\x00 (II)\n\n10-1. 세계경제 동향 (I) ·································································································

## Create Chroma DB

1. Text -> Embbedings
2. `db` 폴더에 데이터 저장
3. DB 초기화
4. `db` 폴더로부터 DB 로드

In [ ]:
persist_directory = 'db'

embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [ ]:
vectordb.persist()
vectordb = None

In [ ]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding)

## Make a retriever

### 결과를 k개 반환

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
docs = retriever.get_relevant_documents("최근 한국의 경제상황은?")

for doc in docs:
    print(doc.metadata["source"])

/content/drive/MyDrive/QUANTIFY/text_files/2023_Korean_Economy_News (1).txt
/content/drive/MyDrive/QUANTIFY/text_files/2023_Korean_Economy_News (1).txt
/content/drive/MyDrive/QUANTIFY/text_files/2023_Korean_Economy_News (1).txt


## Make a chain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# Create the language model instance using GPT-4
llm = ChatOpenAI(model_name='gpt-4')

# Use the llm instance in RetrievalQA.from_chain_type
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

## Query

In [ ]:
query = "한국 경제는 미국에 얼마나 영향받니?"
llm_response = qa(query)
process_llm_response(llm_response)

한국 경제는 미국 경제에 크게 영향을 받습니다. 미국 경제가 성장하면 그 성장은 수출 의존도가 높은 한국 경제에도 긍정적인 영향을 미칩니다. 예를 들어, 미국 경제가 호황일 때에는 한국의 수출이 증가하며, 이는 한국 경제에 흑자를 가져다줍니다. 이창용 한국은행 총재는 미국 경제가 성장하면 그것이 한국 수출에 좋다고 말한 바 있습니다. 그러나 미국과 중국 간의 경제 갈등이 계속될 경우 이는 한국 경제에 불확실성을 가져다줄 수 있습니다.


Sources:
/content/drive/MyDrive/QUANTIFY/text_files/2023_Korean_Economy_News (1).txt
/content/drive/MyDrive/QUANTIFY/text_files/2023_Korean_Economy_News (1).txt
/content/drive/MyDrive/QUANTIFY/text_files/2023_Korean_Economy_News (1).txt


In [ ]:
llm_response

{'query': '한국 경제는 미국에 얼마나 영향받니?',
 'result': '한국 경제는 미국 경제에 크게 영향을 받습니다. 미국 경제가 성장하면 그 성장은 수출 의존도가 높은 한국 경제에도 긍정적인 영향을 미칩니다. 예를 들어, 미국 경제가 호황일 때에는 한국의 수출이 증가하며, 이는 한국 경제에 흑자를 가져다줍니다. 이창용 한국은행 총재는 미국 경제가 성장하면 그것이 한국 수출에 좋다고 말한 바 있습니다. 그러나 미국과 중국 간의 경제 갈등이 계속될 경우 이는 한국 경제에 불확실성을 가져다줄 수 있습니다.',
 'source_documents': [Document(page_content='올해 예상외로 호조를 보인 미국 경제가 내년에도 완만한 성장을 유지하면서 세계 경제 성장세를 이끌 것이란 국내 전문가 분석이 나왔다. 이는 수출 의존도가 높은 한국 경제에도 긍정적인 영향을 줄 것으로 기대된다. 다만 중국 경제가 아직 성장 동력을 되찾을 거라고 기대하기 힘든 상황이고, 세계 교역의 긴장감을 키우는 미·중 갈등도 계속 이어질 가능성이 크기 때문에 한국 경제를 둘러싼 불확실성은 내년에도 여전할 것이란 지적도 제기된다.""美, 내년 세계 경제 버팀목""…韓 호재국립외교원은 최근 내놓은 내년 국제정세전망에서 미국 경제에 대해 ""2024년에도 완만한 성장세를 이어가며 세계 경제 성장의 버팀목이 될 것으로 보인다""며 ""이런 낙관적인 전망의 배경에는 미국 내 개인소비의 견고한 확대를 토대로 한 미국 경제의 연착륙 가능성이 자리 잡고 있다""고 설명했다.민정훈 국립외교원 교수는 전날 정부서울청사에서 열린 관련 브리핑에서 ""이번 연휴 시즌에 쇼핑한 것을 보면 (소비자들이) 생각 외로 지갑을 많이 열었다""며 ""실물 경기에 있어서도 선전하고 있는 만큼 내년 상반기에 미국 경제가 호황으로 갈 가능성이 있다""고 설명했다.실제 마스터카드 스펜딩펄스 보고서에 따르면 미국의 소비 대목인 연말 추수감사절과 크리스마스 기간 중 자동차를 제외한 소매판매는 지난해 같은 기간